# How to call functions with chat models

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`tools` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

Within the `tools` parameter, if the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `tool_choice` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `tool_choice` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `tool_choice` object that has the name of the function and the generated function arguments.

### Overview

This notebook contains the following 2 sections:

- **How to generate function arguments:** Specify a set of functions and use the API to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing functions with model generated arguments.

## How to generate function arguments

In [ ]:
%%capture
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor
!pip install openai
!pip install requests

In [1]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"

In [6]:
with open('C:\\Users\\User\\Desktop\\AUA\\Generative AI\\HW1\\api_key.txt','r') as f:
  openai.api_key = f.read()

### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [7]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


### Basic concepts

Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [24]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': "Sure, may I know the location for which you'd like to know the weather?"}

In [27]:
chat_response.json()

{'id': 'chatcmpl-8qm3AayJgioyyRudu8X0B6DBRxyX9',
 'object': 'chat.completion',
 'created': 1707588452,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_sGlYcSzjqjz2Fue6bGO9KMgh',
      'type': 'function',
      'function': {'name': 'get_current_weather',
       'arguments': '{\n  "location": "Yerevan, Armenia"\n}'}}]},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 121, 'completion_tokens': 20, 'total_tokens': 141},
 'system_fingerprint': None}

In [31]:
type(chat_response.json())

dict

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [26]:
messages.append({"role": "user", "content": "I'm in Yerevan, Armenia."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_sGlYcSzjqjz2Fue6bGO9KMgh',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{\n  "location": "Yerevan, Armenia"\n}'}}]}

In [18]:
loc = assistant_message['tool_calls'][0]['function']['arguments'].split('"')[3]

In [19]:
loc

'Yerevan, Armenia'

In [20]:
import requests

url = "http://api.openweathermap.org/data/2.5/weather"
params = {
    "q": loc,
    "APPID": "3128f76a7551a274746e884fd29a0e8f"
}

response = requests.get(url, params=params)

print(response.status_code)  # Prints the status code of the response
print(response.json())  # Prints the JSON content of the response

200
{'coord': {'lon': 44.5136, 'lat': 40.1811}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 277.24, 'feels_like': 277.24, 'temp_min': 277.24, 'temp_max': 277.24, 'pressure': 1023, 'humidity': 65}, 'visibility': 10000, 'wind': {'speed': 0, 'deg': 0}, 'clouds': {'all': 6}, 'dt': 1707585900, 'sys': {'type': 1, 'id': 8851, 'country': 'AM', 'sunrise': 1707537741, 'sunset': 1707575406}, 'timezone': 14400, 'id': 616052, 'name': 'Yerevan', 'cod': 200}


In [33]:
type(response.json())
# print(response.json()['main']['temp'])

str

In [21]:

response.text

'{"coord":{"lon":44.5136,"lat":40.1811},"weather":[{"id":800,"main":"Clear","description":"clear sky","icon":"01n"}],"base":"stations","main":{"temp":277.24,"feels_like":277.24,"temp_min":277.24,"temp_max":277.24,"pressure":1023,"humidity":65},"visibility":10000,"wind":{"speed":0,"deg":0},"clouds":{"all":6},"dt":1707585900,"sys":{"type":1,"id":8851,"country":"AM","sunrise":1707537741,"sunset":1707575406},"timezone":14400,"id":616052,"name":"Yerevan","cod":200}'

In [22]:
from openai import OpenAI
import json

client = OpenAI(api_key=openai.api_key)

response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  response_format={ "type": "text" },
  messages=[
    {"role": "system", "content": "You are weather assistant"},
    {"role": "user", "content": f"{response.text} based on this information what the temerature in celcius in the given location. Provide short answer."}
  ]
)
print(response.choices[0].message.content)

The temperature in Yerevan is 277.24 Kelvin. To convert it to Celsius, you subtract 273.15 from the Kelvin temperature. 

277.24 K - 273.15 = 4.09°C

So the temperature in Yerevan is approximately 4.09 degrees Celsius.


# END